## Ground site data: model vs obs

This script is used to plot the time series of ground site data from observation and model output.
#### This doesn't work for now

There is a very interesing instruction may be useful for GCv13 output
http://danielrothenberg.com/gcpy/tutorials/xarray_overview.html

In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import matplotlib.dates as mdates
from matplotlib.dates import DateFormatter


In [2]:
# disable chained assignments
pd.options.mode.chained_assignment = None 

### Read observation data in csv file

In [3]:
# file directory
obs_dir = '/glade/work/lixujin/ground'
obs_subdir ='/'
file = obs_dir+obs_subdir+'grdsites_CO_2018.csv'

In [4]:
# read in csv data
df = pd.read_csv(file)#,header=0,index_col=0)
df.head(10)

,DDHH GMT,Boise_Eastman CO ppm,StLukes Meridian CO ppm,StLukes Meridian PM25 ugm3,MBO_CO_ppbv,MBO_PM1_ug/m3,Chico_CO_ppbv,Chico_CO_MDL,Fresno_CO_ppbv,Fresno_CO_MDL,...,Stockton_MDL,Reno_CO,Reno_MDL,King_CO,King_MDL,Denver_CO,Denver_MDL,Ada_CO,Ada_MDL,Missoula_CO
0,7/22/2018 0:00,0.4,0.16,7,103.19,1.6,0.144,0.02,0.164,0.011,...,0.02,0.227,0.02,0.124,0.02,0.222,0.04,0.142,0.02,NaN
1,7/22/2018 1:00,0.4,0.17,9,102.62,1.5,0.152,0.02,0.155,0.011,...,0.02,0.177,0.02,0.135,0.02,0.283,0.04,0.162,0.02,NaN
2,7/22/2018 2:00,0.4,0.17,10,102.87,1.5,0.163,0.02,0.143,0.011,...,0.02,0.178,0.02,0.127,0.02,0.360,0.04,0.177,0.02,NaN
3,7/22/2018 3:00,0.5,0.14,7,104.82,1.6,0.169,0.02,0.162,0.011,...,0.02,0.233,0.02,0.148,0.02,0.413,0.04,0.172,0.02,NaN
4,7/22/2018 4:00,0.4,0.14,4,103.54,1.6,0.164,0.02,0.156,0.011,...,0.02,0.287,0.02,0.187,0.02,0.274,0.04,0.148,0.02,NaN
5,7/22/2018 5:00,0.4,0.12,2,105.87,1.5,0.144,0.02,0.139,0.011,...,0.02,0.438,0.02,0.221,0.02,0.216,0.04,0.149,0.02,NaN
6,7/22/2018 6:00,0.5,0.11,-999,105.71,1.3,0.135,0.02,0.131,0.011,...,0.02,0.306,0.02,0.200,0.02,0.279,0.04,0.129,0.02,NaN
7,7/22/2018 7:00,0.4,0.12,7,106.71,1.2,0.148,0.02,0.128,0.011,...,0.02,0.402,0.02,0.146,0.02,0.337,0.04,0.115,0.02,NaN
8,7/22/2018 8:00,0.4,0.10,6,104.67,1.1,0.137,0.02,0.127,0.011,...,0.02,0.468,0.02,0.132,0.02,0.257,0.04,0.121,0.02,NaN
9,7/22/2018 9:00,0.4,0.10,6,104.38,1.0,0.124,0.02,0.125,0.011,...,0.02,0.426,0.02,0.116,0.02,0.246,0.04,0.106,0.02,NaN


In [5]:
# convert time into panda form to retrieve date later
df['DDHH GMT'] = pd.to_datetime(df['DDHH GMT'])

In [6]:
# fill negative value as missing value 
cols = df.columns[1:]
df.iloc[:,1:][df.iloc[:,1:][cols] < 0] = np.nan

In [7]:
# check missing value
print(df.iloc[:,1:].isnull().sum())
# check value less than 0
#print(df.iloc[:,1:][df.iloc[:,1:] < 0].count())

Boise_Eastman CO ppm            0
StLukes Meridian CO ppm         0
StLukes Meridian PM25 ugm3      0
MBO_CO_ppbv                     0
MBO_PM1_ug/m3                   0
Chico_CO_ppbv                  80
Chico_CO_MDL                   80
Fresno_CO_ppbv                  3
Fresno_CO_MDL                   3
Stockton_CO                    76
Stockton_MDL                   76
Reno_CO                        22
Reno_MDL                        0
King_CO                       232
King_MDL                        0
Denver_CO                      39
Denver_MDL                      0
Ada_CO                        161
Ada_MDL                         0
Missoula_CO                   166
dtype: int64


In [8]:
# Replace data<LoD with LoD
# Since we didn't regularize the data name and MDL, we need to deal with individual sites
# Boise, eastman and Meridian provided by Dan are not good...
# Mt. Bachelor, OR, okay
flt = (df['Chico_CO_ppbv'] < df['Chico_CO_MDL'][0])&(df['Chico_CO_ppbv'] > 0)
df['Chico_CO_ppbv'][flt] = df['Chico_CO_MDL'][0]

# Chico, CA
flt = (df['Chico_CO_ppbv'] < df['Chico_CO_MDL'][0])&(df['Chico_CO_ppbv'] > 0)
df['Chico_CO_ppbv'][flt] = df['Chico_CO_MDL'][0]

# Fresno, CA
flt = (df['Fresno_CO_ppbv'] < df['Fresno_CO_MDL'][0])&(df['Fresno_CO_ppbv'] > 0)
df['Fresno_CO_ppbv'][flt] = df['Fresno_CO_MDL'][0]

# Stockton, CA
flt = (df['Stockton_CO'] < df['Stockton_MDL'][0])&(df['Stockton_CO'] > 0)
df['Stockton_CO'][flt] = df['Stockton_MDL'][0]

# Reno, NV
flt = (df['Reno_CO'] < df['Reno_MDL'][0])&(df['Reno_CO'] > 0)
df['Reno_CO'][flt] = df['Reno_MDL'][0]

# King, WA
flt = (df['King_CO'] < df['King_MDL'][0])&(df['King_CO'] > 0)
df['King_CO'][flt] = df['King_MDL'][0]

# Denver, CO
flt = (df['Denver_CO'] < df['Denver_MDL'][0])&(df['Denver_CO'] > 0)
df['Denver_CO'][flt] = df['Denver_MDL'][0]

# Bosie, ID
flt = (df['Ada_CO'] < df['Ada_MDL'][0])&(df['Ada_CO'] > 0)
df['Ada_CO'][flt] = df['Ada_MDL'][0]

# Missoula, MT, okk

In [9]:
print(df.iloc[:,1:].isnull().sum())

Boise_Eastman CO ppm            0
StLukes Meridian CO ppm         0
StLukes Meridian PM25 ugm3      0
MBO_CO_ppbv                     0
MBO_PM1_ug/m3                   0
Chico_CO_ppbv                  80
Chico_CO_MDL                   80
Fresno_CO_ppbv                  3
Fresno_CO_MDL                   3
Stockton_CO                    76
Stockton_MDL                   76
Reno_CO                        22
Reno_MDL                        0
King_CO                       232
King_MDL                        0
Denver_CO                      39
Denver_MDL                      0
Ada_CO                        161
Ada_MDL                         0
Missoula_CO                   166
dtype: int64


keep in mind that I am using GMT here but not local time. It's okay for WE-CAN paper

In [10]:
# drop columns we don't need this time
#df_data_new  = df_data.drop(['Boise_Eastman CO ppm','StLukes Meridian CO ppm','StLukes Meridian PM25 ugm3','MBO_PM1_ug/m3'],axis=1) #the column number is fixed
#df_data_new  = df_data.drop(['Chico_CO_MDL','Fresno_CO_MDL','Stockton_MDL','Reno_MDL','King_MDL','Denver_MDL','Ada_MDL'],axis=1) #the column number is fixed
df_new  = df.drop(['Boise_Eastman CO ppm','StLukes Meridian CO ppm','StLukes Meridian PM25 ugm3','MBO_PM1_ug/m3',
                             'Chico_CO_MDL','Fresno_CO_MDL','Stockton_MDL','Reno_MDL','King_MDL','Denver_MDL','Ada_MDL'],axis=1) 

# rename Ada as Boise
new_names_dict = {'Ada_CO':'Boise_CO'}
df_new.rename(columns = new_names_dict, inplace=True) #Set inplace eq 1 then the old column names will be replaced

In [11]:
df_new

,DDHH GMT,MBO_CO_ppbv,Chico_CO_ppbv,Fresno_CO_ppbv,Stockton_CO,Reno_CO,King_CO,Denver_CO,Boise_CO,Missoula_CO
0,2018-07-22 00:00:00,103.19,0.144,0.164,0.156,0.227,0.124,0.222,0.142,NaN
1,2018-07-22 01:00:00,102.62,0.152,0.155,0.153,0.177,0.135,0.283,0.162,NaN
2,2018-07-22 02:00:00,102.87,0.163,0.143,0.184,0.178,0.127,0.360,0.177,NaN
3,2018-07-22 03:00:00,104.82,0.169,0.162,0.164,0.233,0.148,0.413,0.172,NaN
4,2018-07-22 04:00:00,103.54,0.164,0.156,0.162,0.287,0.187,0.274,0.148,NaN
...,...,...,...,...,...,...,...,...,...,...
1339,2018-09-15 19:00:00,173.30,0.160,0.163,0.148,0.082,0.162,0.169,0.310,NaN
1340,2018-09-15 20:00:00,168.71,0.160,0.161,0.140,0.089,0.145,0.135,0.283,NaN
1341,2018-09-15 21:00:00,180.31,0.153,0.162,0.143,0.082,0.139,0.118,0.224,NaN
1342,2018-09-15 22:00:00,201.65,0.129,0.162,0.164,0.093,0.142,0.082,0.243,NaN


In [12]:
# convert all data in unit of ppb rather than ppm
df_new.iloc[:,2:] = df_new.iloc[:,2:]*1000

In [13]:
# rerange the columns by geophysical locations; reorder
# WA, MBO, NV, CA, CA, CA, MT, BOISE, CO
df_new = df_new[['DDHH GMT','King_CO','MBO_CO_ppbv','Reno_CO','Fresno_CO_ppbv','Stockton_CO','Chico_CO_ppbv','Denver_CO','Boise_CO','Missoula_CO']]

In [14]:
df_new.head(10)

,DDHH GMT,King_CO,MBO_CO_ppbv,Reno_CO,Fresno_CO_ppbv,Stockton_CO,Chico_CO_ppbv,Denver_CO,Boise_CO,Missoula_CO
0,2018-07-22 00:00:00,124.0,103.19,227.0,164.0,156.0,144.0,222.0,142.0,NaN
1,2018-07-22 01:00:00,135.0,102.62,177.0,155.0,153.0,152.0,283.0,162.0,NaN
2,2018-07-22 02:00:00,127.0,102.87,178.0,143.0,184.0,163.0,360.0,177.0,NaN
3,2018-07-22 03:00:00,148.0,104.82,233.0,162.0,164.0,169.0,413.0,172.0,NaN
4,2018-07-22 04:00:00,187.0,103.54,287.0,156.0,162.0,164.0,274.0,148.0,NaN
5,2018-07-22 05:00:00,221.0,105.87,438.0,139.0,171.0,144.0,216.0,149.0,NaN
6,2018-07-22 06:00:00,200.0,105.71,306.0,131.0,149.0,135.0,279.0,129.0,NaN
7,2018-07-22 07:00:00,146.0,106.71,402.0,128.0,135.0,148.0,337.0,115.0,NaN
8,2018-07-22 08:00:00,132.0,104.67,468.0,127.0,118.0,137.0,257.0,121.0,NaN
9,2018-07-22 09:00:00,116.0,104.38,426.0,125.0,103.0,124.0,246.0,106.0,NaN


In [15]:
# rename columns
new_names_dict = {'King_CO':'Seattle, WA', 
                  'MBO_CO_ppbv': 'Mt. Bachelor, OR', 
                  'Reno_CO': 'Reno, NV', 
                  'Fresno_CO_ppbv': 'Fresno, CA',
                  'Stockton_CO': 'Stockton, CA',
                  'Chico_CO_ppbv':'Chico, CA',
                  'Denver_CO': 'Denver, CO',
                  'Boise_CO': 'Boise, ID',
                  'Missoula_CO': 'Missoula, MT'}
df_obs = df_new.rename(columns = new_names_dict, inplace=False) #Set inplace eq 1 then the old column names will be replaced
df_obs.head(10)

,DDHH GMT,"Seattle, WA","Mt. Bachelor, OR","Reno, NV","Fresno, CA","Stockton, CA","Chico, CA","Denver, CO","Boise, ID","Missoula, MT"
0,2018-07-22 00:00:00,124.0,103.19,227.0,164.0,156.0,144.0,222.0,142.0,NaN
1,2018-07-22 01:00:00,135.0,102.62,177.0,155.0,153.0,152.0,283.0,162.0,NaN
2,2018-07-22 02:00:00,127.0,102.87,178.0,143.0,184.0,163.0,360.0,177.0,NaN
3,2018-07-22 03:00:00,148.0,104.82,233.0,162.0,164.0,169.0,413.0,172.0,NaN
4,2018-07-22 04:00:00,187.0,103.54,287.0,156.0,162.0,164.0,274.0,148.0,NaN
5,2018-07-22 05:00:00,221.0,105.87,438.0,139.0,171.0,144.0,216.0,149.0,NaN
6,2018-07-22 06:00:00,200.0,105.71,306.0,131.0,149.0,135.0,279.0,129.0,NaN
7,2018-07-22 07:00:00,146.0,106.71,402.0,128.0,135.0,148.0,337.0,115.0,NaN
8,2018-07-22 08:00:00,132.0,104.67,468.0,127.0,118.0,137.0,257.0,121.0,NaN
9,2018-07-22 09:00:00,116.0,104.38,426.0,125.0,103.0,124.0,246.0,106.0,NaN


So far, we have prepared the data. Time range could be customed if necessary.

In [16]:
# create daily data
# resampble data, average data into daily
df_obs_daily = df_obs.resample('D', on='DDHH GMT').mean()

# create diurnal data
# create a tmp variable to avoid messing up with df_new
tmp = df_obs
# add a column to show hour of the day
tmp['HH'] = tmp['DDHH GMT'].map(lambda x: x.strftime("%H:%M"))
# group by the hour to get dirunal pattern
df_obs_diurnal = tmp.groupby('HH').mean()

### Read model data in netcdf file

In [17]:
# xarray allows us to read in any NetCDF file, the format of most GEOS-Chem diagnostics, as an xarray Dataset
import xarray as xr
import datetime
from dateutil.rrule import rrule, DAILY
import sys
import numpy as np
import pandas as pd

#### Define a function to read in sites data

##### Read in model data

In [18]:
def get_site_data_V13(files_pfx,start_date,end_date):
    fl_sfx = '_0000z.nc4'
    site = files_pfx.split('_')[1].split('.')[0]
    
    # use a normal list to save data later
    co_mod_list = []
    date_mod_list = []
    
    for day in rrule(DAILY, dtstart=start_date, until=end_date):
        day_fmt = datetime.datetime.strftime(day, '%Y%m%d')
        src_fl = '{0}{1}{2}'.format(files_pfx, day_fmt, fl_sfx)
        
        print(src_fl)
        
        ds = xr.open_dataset(src_fl)
        tmp = ds['SpeciesConc_CO'][:,0,0,0].values.tolist()
        # create data
        co_mod_list.append(tmp)
        # craete date
        date_mod_list.append(day_fmt)
        #sys.exit()
        
    # output the shape of the list
    np.shape(co_mod_list)

    # convert the data list into dataframe
    # convert mol/mol to ppb
    co_mod_df = pd.DataFrame(co_mod_list, index = date_mod_list)*1e9
    co_mod_df.head(10)

    # create diurnal pattern
    co_mod_daily = pd.DataFrame(co_mod_df.mean(axis=1),columns=[site])
    
    # return the diurnal pattern
    return co_mod_daily

In [19]:
from os.path import exists
def get_site_data_V12(file_dir, start_date, end_date, MBO, compound):
    # create date lists
    date_list = []
    for day in rrule(DAILY, dtstart=start_date, until=end_date):
        date_list = date_list + [datetime.datetime.strftime(day, '%Y-%m-%d')]

    if MBO == True:
        station_num = 10
        data_tmp = []
        for day in rrule(DAILY, dtstart=start_date, until=end_date):
            day_fmt = datetime.datetime.strftime(day, '%Y%m%d')
            src_fl = '{0}{1}{2}'.format(file_dir+'stations.', day_fmt, '.nc')
            if exists(src_fl):
                ds = xr.open_dataset(src_fl)
                tmp = ds['IJ_AVG_S_' + compound][:,0,0].values.tolist()
                data_tmp = data_tmp + tmp
            
            
        data_tmp = np.array(data_tmp).reshape(int(len(data_tmp)/station_num), station_num)
        #Seattle, King-North Beacon Hill
        data_mod_king = data_tmp[:,0]
        data_mod_king =data_mod_king.reshape(int(len(data_mod_king)/24), 24).mean(1)

        #Portland, Multnomah, no good
        data_mod_portland = data_tmp[:,1]
        data_mod_portland =data_mod_portland.reshape(int(len(data_mod_portland)/24), 24).mean(1)

        #Boise, Ada-Meridian
        data_mod_ada = data_tmp[:,2]
        data_mod_ada =data_mod_ada.reshape(int(len(data_mod_ada)/24), 24).mean(1)

        #Denver, Denver-sunnyside
        data_mod_denver = data_tmp[:,3]
        data_mod_denver =data_mod_denver.reshape(int(len(data_mod_denver)/24), 24).mean(1)
        
        #Stockton, San Joaquin
        data_mod_sanjoquin = data_tmp[:,4]
        data_mod_sanjoquin =data_mod_sanjoquin.reshape(int(len(data_mod_sanjoquin)/24), 24).mean(1)
        
        #Fresno, Fresno-3727 N first str
        data_mod_fresno = data_tmp[:,5]
        data_mod_fresno =data_mod_fresno.reshape(int(len(data_mod_fresno)/24), 24).mean(1)

        #Reno, Washoe-98-02 river rock st.
        data_mod_washoe = data_tmp[:,7]
        data_mod_washoe =data_mod_washoe.reshape(int(len(data_mod_washoe)/24), 24).mean(1)
        #Chico, Butte
        data_mod_butte = data_tmp[:,8]
        data_mod_butte =data_mod_butte.reshape(int(len(data_mod_butte)/24), 24).mean(1)
        #Missoula, missoula-UM
        data_mod_missoula = data_tmp[:,9]
        data_mod_missoula =data_mod_missoula.reshape(int(len(data_mod_missoula)/24), 24).mean(1)
        
        data = {'Seattle, WA': data_mod_king,
                'Boise, ID': data_mod_ada,
                'Denver, CO': data_mod_denver,
                'Stockton, CA':  data_mod_sanjoquin,
                'Fresno, CA': data_mod_fresno,
                'Reno, NV': data_mod_washoe,
                'Chico, CA': data_mod_butte,
                'Missoula, MT': data_mod_missoula}
    else:
        data_mod_MBO = []
        # create normal list
        for day in rrule(DAILY, dtstart=start_date, until=end_date):
            day_fmt = datetime.datetime.strftime(day, '%Y%m%d')
            src_fl = '{0}{1}{2}'.format(file_dir+'ts', day_fmt, '.nc')
            ds = xr.open_dataset(src_fl)
            tmp = ds['IJ_AVG_S_' + compound][:,0,0,16].values.tolist()
            data_mod_MBO = data_mod_MBO + tmp

        data_mod_MBO=np.array(data_mod_MBO)
        #MBO
        data_mod_MBO =data_mod_MBO.reshape(int(len(data_mod_MBO)/24), 24).mean(1)
        
        data = {'Mt. Bachelor, OR':data_mod_MBO}
        
    df_mod_sites = pd.DataFrame(data, index = date_list)
    return df_mod_sites

In [20]:
'''
# function to print out correlations for numpy data (obsolete)
def correlation(A, B):
    return np.ma.corrcoef(A,B)[0][1]
'''

'\n# function to print out correlations for numpy data (obsolete)\ndef correlation(A, B):\n    return np.ma.corrcoef(A,B)[0][1]\n'

In [21]:
# CO
file_dir_gfas = '/glade/scratch/lixujin/rundirs/GC12.5.0/geosfp_025x03125_tropchem_na_gfas_sf_mami/stations/'
file_dir_threegfas = '/glade/scratch/lixujin/rundirs/GC12.5.0/geosfp_025x03125_tropchem_na_3Xgfas_sf_mami/stations/'
file_dir_nobb = '/glade/scratch/lixujin/rundirs/GC12.5.0/geosfp_025x03125_tropchem_na_nobb_v2/stations/'

start_date = datetime.date(2018,7,22)
end_date = datetime.date(2018,9,14)

df_mod_CO_base = pd.concat([get_site_data_V12(file_dir_gfas, start_date, end_date, MBO=True, compound='CO'),
                            get_site_data_V12(file_dir_gfas, start_date, end_date, MBO=False, compound='CO')],  axis=1)

df_mod_CO_threegfas = pd.concat([get_site_data_V12(file_dir_threegfas, start_date, end_date, MBO=True, compound='CO'),
                                 get_site_data_V12(file_dir_threegfas, start_date, end_date, MBO=False, compound='CO')],  axis=1)

df_mod_CO_nobb = pd.concat([get_site_data_V12(file_dir_nobb, start_date, end_date, MBO=True, compound='CO'),
                                 get_site_data_V12(file_dir_nobb, start_date, end_date, MBO=False, compound='CO')],  axis=1)

In [22]:
'''
# PM2.5
def PM_calcualtion(file_dir, start_date, end_date):
    df_mod_NH4 = pd.concat([get_site_data_V12(file_dir, start_date, end_date, MBO=True, compound='NH4'),
                            get_site_data_V12(file_dir, start_date, end_date, MBO=False, compound='NH4')],  axis=1)
    df_mod_NIT = pd.concat([get_site_data_V12(file_dir, start_date, end_date, MBO=True, compound='NIT'),
                            get_site_data_V12(file_dir, start_date, end_date, MBO=False, compound='NIT')],  axis=1)
    df_mod_SO4 = pd.concat([get_site_data_V12(file_dir, start_date, end_date, MBO=True, compound='SO4'),
                            get_site_data_V12(file_dir, start_date, end_date, MBO=False, compound='SO4')],  axis=1)
    df_mod_BCPI = pd.concat([get_site_data_V12(file_dir, start_date, end_date, MBO=True, compound='BCPI'),
                            get_site_data_V12(file_dir, start_date, end_date, MBO=False, compound='BCPI')],  axis=1)
    df_mod_BCPO = pd.concat([get_site_data_V12(file_dir, start_date, end_date, MBO=True, compound='BCPO'),
                            get_site_data_V12(file_dir, start_date, end_date, MBO=False, compound='BCPO')],  axis=1)
    df_mod_OCPI = pd.concat([get_site_data_V12(file_dir, start_date, end_date, MBO=True, compound='OCPI'),
                            get_site_data_V12(file_dir, start_date, end_date, MBO=False, compound='OCPI')],  axis=1)
    df_mod_OCPO = pd.concat([get_site_data_V12(file_dir, start_date, end_date, MBO=True, compound='OCPO'),
                            get_site_data_V12(file_dir, start_date, end_date, MBO=False, compound='OCPO')],  axis=1)
    df_mod_SOAS = pd.concat([get_site_data_V12(file_dir, start_date, end_date, MBO=True, compound='SOAS'),
                            get_site_data_V12(file_dir, start_date, end_date, MBO=False, compound='SOAS')],  axis=1)
    df_mod_DST1 = pd.concat([get_site_data_V12(file_dir, start_date, end_date, MBO=True, compound='DST1'),
                            get_site_data_V12(file_dir, start_date, end_date, MBO=False, compound='DST1')],  axis=1)
    df_mod_DST2 = pd.concat([get_site_data_V12(file_dir, start_date, end_date, MBO=True, compound='DST2'),
                            get_site_data_V12(file_dir, start_date, end_date, MBO=False, compound='DST2')],  axis=1)
    df_mod_SALA = pd.concat([get_site_data_V12(file_dir, start_date, end_date, MBO=True, compound='SALA'),
                            get_site_data_V12(file_dir, start_date, end_date, MBO=False, compound='SALA')],  axis=1)

    df_mod_pm_1 = df_mod_NH4.add(df_mod_NIT).add(df_mod_SO4).mul(1.33)
    df_mod_pm_2 = df_mod_BCPI.add(df_mod_BCPO)
    df_mod_pm_3 = df_mod_OCPI.mul(1.16).add(df_mod_OCPO).mul(2.1)
    df_mod_pm_4 = df_mod_SOAS.mul(1.16).add(df_mod_DST1)
    df_mod_pm_5 = df_mod_DST2.mul(0.38)
    df_mod_pm_6 = df_mod_SALA.mul(1.86)
    
    df_mod_pm = df_mod_pm_1.add(df_mod_pm_2).add(df_mod_pm_3).add(df_mod_pm_4).add(df_mod_pm_5).add(df_mod_pm_6)
    
    return df_mod_pm
df_mod_PM = PM_calcualtion(file_dir, start_date, end_date)
'''


"\n# PM2.5\ndef PM_calcualtion(file_dir, start_date, end_date):\n    df_mod_NH4 = pd.concat([get_site_data_V12(file_dir, start_date, end_date, MBO=True, compound='NH4'),\n                            get_site_data_V12(file_dir, start_date, end_date, MBO=False, compound='NH4')],  axis=1)\n    df_mod_NIT = pd.concat([get_site_data_V12(file_dir, start_date, end_date, MBO=True, compound='NIT'),\n                            get_site_data_V12(file_dir, start_date, end_date, MBO=False, compound='NIT')],  axis=1)\n    df_mod_SO4 = pd.concat([get_site_data_V12(file_dir, start_date, end_date, MBO=True, compound='SO4'),\n                            get_site_data_V12(file_dir, start_date, end_date, MBO=False, compound='SO4')],  axis=1)\n    df_mod_BCPI = pd.concat([get_site_data_V12(file_dir, start_date, end_date, MBO=True, compound='BCPI'),\n                            get_site_data_V12(file_dir, start_date, end_date, MBO=False, compound='BCPI')],  axis=1)\n    df_mod_BCPO = pd.concat([get_site_da

In [23]:
df_obs_daily = df_obs_daily.reindex(['Seattle, WA','Boise, ID','Missoula, MT', 
                                    'Mt. Bachelor, OR', 'Reno, NV', 'Denver, CO',
                                    'Chico, CA', 'Stockton, CA', 'Fresno, CA'], axis=1)[:-1]
df_mod_CO_base = df_mod_CO_base.reindex(['Seattle, WA','Boise, ID','Missoula, MT', 
                                    'Mt. Bachelor, OR', 'Reno, NV', 'Denver, CO',
                                    'Chico, CA', 'Stockton, CA', 'Fresno, CA'], axis=1)
df_mod_CO_threegfas = df_mod_CO_threegfas.reindex(['Seattle, WA','Boise, ID','Missoula, MT', 
                                    'Mt. Bachelor, OR', 'Reno, NV', 'Denver, CO',
                                    'Chico, CA', 'Stockton, CA', 'Fresno, CA'], axis=1)
df_mod_CO_nobb = df_mod_CO_nobb.reindex(['Seattle, WA','Boise, ID','Missoula, MT', 
                                    'Mt. Bachelor, OR', 'Reno, NV', 'Denver, CO',
                                    'Chico, CA', 'Stockton, CA', 'Fresno, CA'], axis=1)

In [24]:
df_obs_daily.index

DatetimeIndex(['2018-07-22', '2018-07-23', '2018-07-24', '2018-07-25',
               '2018-07-26', '2018-07-27', '2018-07-28', '2018-07-29',
               '2018-07-30', '2018-07-31', '2018-08-01', '2018-08-02',
               '2018-08-03', '2018-08-04', '2018-08-05', '2018-08-06',
               '2018-08-07', '2018-08-08', '2018-08-09', '2018-08-10',
               '2018-08-11', '2018-08-12', '2018-08-13', '2018-08-14',
               '2018-08-15', '2018-08-16', '2018-08-17', '2018-08-18',
               '2018-08-19', '2018-08-20', '2018-08-21', '2018-08-22',
               '2018-08-23', '2018-08-24', '2018-08-25', '2018-08-26',
               '2018-08-27', '2018-08-28', '2018-08-29', '2018-08-30',
               '2018-08-31', '2018-09-01', '2018-09-02', '2018-09-03',
               '2018-09-04', '2018-09-05', '2018-09-06', '2018-09-07',
               '2018-09-08', '2018-09-09', '2018-09-10', '2018-09-11',
               '2018-09-12', '2018-09-13', '2018-09-14'],
              dtype

In [40]:
from sklearn.metrics import mean_squared_error
import matplotlib.font_manager as font_manager
import math
# ============
# Plot setting
# ============
linewidth = 3


# =================================
# initialzie data for later saveout
# =================================
# site
site_names = []
# =====================
# statistical variables
# =====================
# entire season
MB_base_entire = []
RMSE_base_entire = []
R_base_entire = []
MB_threegfas_entire = []
RMSE_threegfas_entire = []
R_threegfas_entire = []
# BB-impacted
MB_base_smoky = []
RMSE_base_smoky = []
R_base_smoky = []
MB_threegfas_smoky = []
RMSE_threegfas_smoky = []
R_threegfas_smoky = []
# least BB-impacted
MB_base_nonsmoky = []
RMSE_base_nonsmoky = []
R_base_nonsmoky = []
MB_threegfas_nonsmoky = []
RMSE_threegfas_nonsmoky = []
R_threegfas_nonsmoky = []

# walk through each site
fig, axes = plt.subplots(3, 3, figsize=(18,15),sharex=True)
for cols, ax in zip(df_obs_daily.columns[0:], axes.flatten()):
    site_names.append(cols)
    # ============
    # data process
    # ============
    # Making model values as False if observations are.
    # creating bool series False for NaN values 
    bool_series = pd.notnull(df_obs_daily[cols])
    df_mod_CO_base[cols] = df_mod_CO_base[cols][bool_series]
    df_mod_CO_threegfas[cols] = df_mod_CO_threegfas[cols][bool_series]
    df_mod_CO_nobb[cols] = df_mod_CO_nobb[cols][bool_series]
    
    # Define BB-impacted days as when the modeled CO daily enhancement is increased by more than 20% 
    # creating bool series for smoky and non-smoky days
    bool_series_smoky = (df_mod_CO_base[cols]-df_mod_CO_nobb[cols])/df_mod_CO_nobb[cols] >= 0.2
    bool_series_nonsmoky = (df_mod_CO_base[cols]-df_mod_CO_nobb[cols])/df_mod_CO_nobb[cols] < 0.2

    df_obs_daily_smoky = df_obs_daily[cols][bool_series_smoky]
    df_obs_daily_nonsmoky = df_obs_daily[cols][bool_series_nonsmoky]
    df_mod_CO_base_smoky = df_mod_CO_base[cols][bool_series_smoky]
    df_mod_CO_base_nonsmoky = df_mod_CO_base[cols][bool_series_nonsmoky]
    df_mod_CO_threegfas_smoky = df_mod_CO_threegfas[cols][bool_series_smoky]
    df_mod_CO_threegfas_nonsmoky = df_mod_CO_threegfas[cols][bool_series_nonsmoky]
    
    # Develop data for specific time period (obsolete)
    # specific time, i.e., july 24th to August 11th
    start_date = datetime.date(2018,7,25)
    end_date = datetime.date(2018,8,11)
    date_custom = []
    for day in rrule(DAILY, dtstart=start_date, until=end_date):
        day_fmt = datetime.datetime.strftime(day, '%Y-%m-%d')
        date_custom = date_custom + [day_fmt]
        date_custom_obs = pd.to_datetime(date_custom)
    df_obs_daily_spec = df_obs_daily[cols][date_custom_obs]
    df_mod_CO_base_spec = df_mod_CO_base[cols][date_custom]
    df_mod_CO_threegfas_spec = df_mod_CO_threegfas[cols][date_custom]
    
    # Select data points when negative bias of CO > 100 ppb for smoky days.
    bool_series_smoky_100bias = (df_obs_daily_smoky-df_mod_CO_base_smoky)>100
    df_obs_daily_smoky_100bias = df_obs_daily_smoky[bool_series_smoky_100bias] 
    df_mod_CO_base_smoky_100bias = df_mod_CO_base_smoky[bool_series_smoky_100bias]
    df_mod_CO_threegfas_smoky_100bias = df_mod_CO_threegfas_smoky[bool_series_smoky_100bias]

    # =====================
    # statistical analysis
    # =====================
    # for the entire season
    # print("This is for entire campaign.")
    data_obs_daily = df_obs_daily[cols].dropna()
    data_mod_CO_base = df_mod_CO_base[cols].dropna()
    data_mod_CO_threegfas = df_mod_CO_threegfas[cols].dropna()
    
    MB_base_entire.append(-np.mean(data_obs_daily-data_mod_CO_base))
    RMSE_base_entire.append(math.sqrt(mean_squared_error(data_obs_daily, data_mod_CO_base)))
    R_base_entire.append(np.ma.corrcoef(data_obs_daily, data_mod_CO_base)[0,1])
    
    MB_threegfas_entire.append(-np.mean(data_obs_daily-data_mod_CO_threegfas))
    RMSE_threegfas_entire.append(math.sqrt(mean_squared_error(data_obs_daily, data_mod_CO_threegfas)))
    R_threegfas_entire.append(np.ma.corrcoef(data_obs_daily, data_mod_CO_threegfas)[0,1])

    # for the smoky data
    # print("This is for smoky data.")
    data_obs_daily_smoky = df_obs_daily_smoky.dropna()
    data_mod_CO_base_smoky = df_mod_CO_base_smoky.dropna()
    data_mod_CO_threegfas_smoky = df_mod_CO_threegfas_smoky.dropna()
    
    MB_base_smoky.append(-np.mean(data_obs_daily_smoky-data_mod_CO_base_smoky))
    RMSE_base_smoky.append(math.sqrt(mean_squared_error(data_obs_daily_smoky, data_mod_CO_base_smoky)))
    R_base_smoky.append(np.ma.corrcoef(data_obs_daily_smoky, data_mod_CO_base_smoky)[0,1])
    
    MB_threegfas_smoky.append(-np.mean(data_obs_daily_smoky-data_mod_CO_threegfas_smoky))
    RMSE_threegfas_smoky.append(math.sqrt(mean_squared_error(data_obs_daily_smoky, data_mod_CO_threegfas_smoky)))
    R_threegfas_smoky.append(np.ma.corrcoef(data_obs_daily_smoky, data_mod_CO_threegfas_smoky)[0,1])

    # for the nonsmoky data
    # print("This is for nonsmoky data.")
    data_obs_daily_nonsmoky = df_obs_daily_nonsmoky.dropna()
    data_mod_CO_base_nonsmoky = df_mod_CO_base_nonsmoky.dropna()
    data_mod_CO_threegfas_nonsmoky = df_mod_CO_threegfas_nonsmoky.dropna()
    
    MB_base_nonsmoky.append(-np.mean(data_obs_daily_nonsmoky-data_mod_CO_base_nonsmoky))
    RMSE_base_nonsmoky.append(math.sqrt(mean_squared_error(data_obs_daily_nonsmoky, data_mod_CO_base_nonsmoky)))
    R_base_nonsmoky.append(np.ma.corrcoef(data_obs_daily_nonsmoky, data_mod_CO_base_nonsmoky)[0,1])
    
    MB_threegfas_nonsmoky.append(-np.mean(data_obs_daily_nonsmoky-data_mod_CO_threegfas_nonsmoky))
    RMSE_threegfas_nonsmoky.append(math.sqrt(mean_squared_error(data_obs_daily_nonsmoky, data_mod_CO_threegfas_nonsmoky)))
    R_threegfas_nonsmoky.append(np.ma.corrcoef(data_obs_daily_nonsmoky, data_mod_CO_threegfas_nonsmoky)[0,1])

    # test for bkg values (defined the bkg as the lowest 5th of the non smoky days
    print(np.min(abs(-data_obs_daily_nonsmoky+data_mod_CO_base_nonsmoky)))
    
    '''
    # specific dates
    print('customized days')
    if cols == 'Fresno, CA':
        # correlation
        print(np.corrcoef(df_obs_daily_spec, df_mod_CO_base_spec)[0,1],
              np.corrcoef(df_obs_daily_spec, df_mod_CO_threegfas_spec)[0,1])
        # mean bias
        print(np.mean(df_obs_daily_spec-df_mod_CO_base_spec), 
              np.mean(df_obs_daily_spec-df_mod_CO_threegfas_spec))

        # Root Mean Square Error
        MSE_base  = mean_squared_error(df_obs_daily_spec, df_mod_CO_base_spec)
        RMSE_base = math.sqrt(MSE_base)
        MSE_threegfas  = mean_squared_error(df_obs_daily_spec, df_mod_CO_threegfas_spec)
        RMSE_threegfas = math.sqrt(MSE_threegfas)
        print(RMSE_base,
              RMSE_threegfas)
    '''
    
    
    # ===============
    # plot the figure
    # ===============
    ax.plot(df_obs_daily.index, df_obs_daily[cols], 
            color='black',  label='Observation', linewidth=linewidth)
    ax.plot(df_obs_daily.index, df_mod_CO_base[cols], 
            color = 'blue', label='GEOS-Chem + GFAS', linewidth=linewidth)
    ax.plot(df_obs_daily.index, df_mod_CO_threegfas[cols], 
            ':', color = 'grey', label='GEOS-Chem + 3×GFAS', linewidth=linewidth)
    ax.plot(df_obs_daily.index, df_mod_CO_nobb[cols],
            ':', color='hotpink', label='GEOS-Chem + noBB', linewidth=linewidth)
    
    if cols == 'Missoula, MT':
        font = font_manager.FontProperties(size=10,
                                           weight='semibold')
        ax.legend(loc="upper right", prop=font)
    
    # fill out the smoky days
    ax.fill_between(df_obs_daily.index, 0, 1, where=bool_series_smoky, alpha=0.2, transform=ax.get_xaxis_transform())

    # Spacing Out
    plt.subplots_adjust(wspace=.10, hspace=.10)
    ax.set_title(cols, fontsize=15) 
    
    # xrange and yrange
    if cols == 'Seattle, WA': ax.set_ylim([0, 1000])
    if cols == 'Chico, CA': ax.set_ylim([0, 800])
    if cols == 'Stockton, CA': ax.set_ylim([0, 800])


    ax.tick_params(axis='both', labelsize=12)
        
    # Use a more precise date string for the x axis locations in the toolbar.
    ax.fmt_xdata = mdates.DateFormatter('%m-%d')
    
    # Define the date format
    date_form = DateFormatter("%m-%d")
    ax.xaxis.set_major_formatter(date_form)

    # Rotate and align the tick labels so they look better.
    fig.autofmt_xdate()
# texts
fig.text(0.09,0.5, "CO mixing ratio (ppb)", ha="center", va="center", rotation=90, fontsize=18)
fig.text(0.3,0.91, 'CO daily pattern over western US (WE-CAN)', fontsize=25)
fig.savefig('./figure/grd_dailypattern.png')
plt.close() #where f is the figure

19.362391153971345
20.094971021016434
2.906787122660319
8.224775911966958
9.583048502604171
1.0756969451904297
37.58309853595236
18.88151417607847
29.75326633453369


In [26]:
# entire season
data_entire = {'MB (base)': np.round(np.array(MB_base_entire),2),
               'MB (3 × GFAS)': np.round(np.array(MB_threegfas_entire),2),
               'RMSE (base)': np.round(np.array(RMSE_base_entire),2),
               'RMSE (3 × GFAS)': np.round(np.array(RMSE_threegfas_entire),2),
               'R (base)': np.round(np.array(R_base_entire),2),
               'R (3 × GFAS)': np.round(np.array(R_threegfas_entire),2)
              }
df_entire = pd.DataFrame(data = data_entire, index=site_names)

# smoky days
data_smoky = {'MB (base)': np.round(np.array(MB_base_smoky),2),
               'MB (3 × GFAS)': np.round(np.array(MB_threegfas_smoky),2),
               'RMSE (base)': np.round(np.array(RMSE_base_smoky),2),
               'RMSE (3 × GFAS)': np.round(np.array(RMSE_threegfas_smoky),2),
               'R (base)': np.round(np.array(R_base_smoky),2),
               'R (3 × GFAS)': np.round(np.array(R_threegfas_smoky),2)}
df_smoky = pd.DataFrame(data = data_smoky, index=site_names)

# nonsmoky days
data_nonsmoky = {'MB (base)': np.round(np.array(MB_base_nonsmoky),2),
               'MB (3 × GFAS)': np.round(np.array(MB_threegfas_nonsmoky),2),
               'RMSE (base)': np.round(np.array(RMSE_base_nonsmoky),2),
               'RMSE (3 × GFAS)': np.round(np.array(RMSE_threegfas_nonsmoky),2),
               'R (base)': np.round(np.array(R_base_nonsmoky),2),
               'R (3 × GFAS)': np.round(np.array(R_threegfas_nonsmoky),2)}
df_nonsmoky = pd.DataFrame(data = data_nonsmoky, index=site_names)


In [27]:
# saving the dataframe
df_entire.to_csv('./output_data/grd_timeseries_entire.csv')
df_smoky.to_csv('./output_data/grd_timeseries_smoky.csv')
df_nonsmoky.to_csv('./output_data/grd_timeseries_nonsmoky.csv')

In [28]:
# calcualte background value

In [29]:
df_obs_daily.quantile(0.05) - df_mod_CO_base.quantile(0.05)

Seattle, WA        -43.958447
Boise, ID           57.054892
Missoula, MT        18.281254
Mt. Bachelor, OR    17.997981
Reno, NV            24.567765
Denver, CO          31.043563
Chico, CA           74.283879
Stockton, CA        41.535043
Fresno, CA          53.758598
Name: 0.05, dtype: float64

In [31]:
# create fl_prx_list
# sites
sites = ['Seattle','Portland','Boise','Denver','Stockton','Fresno','Reno','Chico','Missoula','MBO']
#sites = ['Seattle','Portland']

# create normal list
fl_prx_list = []
num = 0
for ss in sites:
    num = num + 1
    tmp_fl_prx = '/glade/scratch/lixujin/rundirs/GCv13/OutputDir/GEOSChem.SpeciesConcSubset' + str(num) + '.hourly_' + ss + '.'
    fl_prx_list.append(tmp_fl_prx)
    
# define start date and end date
strt_dt = datetime.date(2018,7,17)
end_dt = datetime.date(2018,8,20)
#end_dt =datetime.date(2018,7,18)

In [32]:
%%time
# create a empty dataframe
df_final = pd.DataFrame()
# read in data
num = 0
for fl_prx in fl_prx_list:
    df_tmp = get_site_data(fl_prx, strt_dt, end_dt)
    df_final = pd.concat([df_final, df_tmp], axis=1)

NameError: name 'get_site_data' is not defined

Process data before analyzing it may save some time

In [33]:
# Reorder columns
df_final = df_final[['Seattle','MBO','Reno','Fresno','Stockton','Chico','Denver','Boise','Missoula']]
df_final.head(10)
# rename columns
new_names_dict = {'Seattle':'Seattle, WA', 
                  'MBO': 'Mt.Bachelor, OR', 
                  'Reno': 'Reno, NV', 
                  'Fresno': 'Fresno, CA',
                  'Stockton': 'Stockton, CA',
                  'Chico':'Chico, CA',
                  'Denver': 'Denver, CO',
                  'Boise': 'Boise, ID',
                  'Missoula': 'Missoula, MT'}
df_mod_diurnal = df_final.rename(columns = new_names_dict, inplace=False) #Set inplace eq 1 then the old column names will be replaced

KeyError: "None of [Index(['Seattle', 'MBO', 'Reno', 'Fresno', 'Stockton', 'Chico', 'Denver',\n       'Boise', 'Missoula'],\n      dtype='object')] are in the [columns]"

##### Save model data

In [ ]:
file = '/glade/work/lixujin/PYTHON/PROJECT/WE-CAN/df_diurnal_mod.csv'
df_mod_diurnal.to_csv(file)

##### read prepared model data

In [ ]:
# read in csv data
import pandas as pd
df_mod_diurnal = pd.read_csv(file,index_col=0)#,header=0,index_col=0)
df_mod_diurnal.head(10)

##### Do the plotting

In [ ]:
# test
fig, axes = plt.subplots(3, 3, figsize=(20,18),sharex=True)
for cols, ax in zip(df_obs.columns[1:], axes.flatten()):
    ax.plot(df_obs['DDHH GMT'], df_obs[cols])
    # Spacing Out
    plt.subplots_adjust(wspace=.10, hspace=.10)
    ax.set_title(cols) #Seattle, WA; Mt.Bachelor, OR; Reno, NV; Fresno, CA; Stockton, CA; Chico, CA; Denver, CO; Boise, ID; Missoula, MT 
    
    # Rotate and align the tick labels so they look better.
    fig.autofmt_xdate()
    # Use a more precise date string for the x axis locations in the toolbar.
    ax.fmt_xdata = mdates.DateFormatter('%Y-%m-%d')
    

In [ ]:
# daily pattern plot
fig, axes = plt.subplots(3, 3, figsize=(24,18),sharex=True)
for cols, ax in zip(df_obs_daily.columns, axes.flatten()):
    ax.plot(df_obs_daily.index, df_obs_daily[cols])
    
    # Spacing Out
    plt.subplots_adjust(wspace=.10, hspace=.10)

    #Seattle, WA; Mt.Bachelor, OR; Reno, NV; Fresno, CA; Stockton, CA; Chico, CA; Denver, CO; Boise, ID; Missoula, MT 
    ax.set_title(cols) 
    
    # Rotate and align the tick labels so they look better.
    fig.autofmt_xdate()
    # Use a more precise date string for the x axis locations in the toolbar.
    ax.fmt_xdata = mdates.DateFormatter('%m-%d')

In [ ]:
# diurnal pattern plot
fig, axes = plt.subplots(3, 3, figsize=(24,20),sharex=True)
for cols, ax in zip(df_obs_diurnal.columns, axes.flatten()):
    ax.plot(df_obs_diurnal.index, df_obs_diurnal[cols])
    ax.plot(df_mod_diurnal.index, df_mod_diurnal[cols])

    # Spacing Out
    plt.subplots_adjust(wspace=.10, hspace=.10)
    ax.set_title(cols) #Seattle, WA; Mt.Bachelor, OR; Reno, NV; Fresno, CA; Stockton, CA; Chico, CA; Denver, CO; Boise, ID; Missoula, MT 
    
    # Rotate and align the tick labels so they look better.
    fig.autofmt_xdate()
    # Use a more precise date string for the x axis locations in the toolbar.
    ax.fmt_xdata = mdates.DateFormatter('%m-%d')